<a href="https://colab.research.google.com/github/qngyn/BigData_Project1/blob/main/Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 199 kB 38.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=ed139763009077e48c8ca0202923c55a9df7bcfc372b7d899f3af0c5613696d3
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [6]:
pip install pyodbc 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 472 kB 7.1 MB/s 


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyodbc 
import pandas as pd 
import pyspark.sql.functions as sf 

In [8]:
spark = SparkSession.builder.config('spark.driver.memory', '16g').getOrCreate()

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
path ='/content/drive/MyDrive/Dataset/20220401.json'

In [11]:
df = spark.read.json(path)

In [12]:
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- _index: string (nullable = true)
 |-- _score: long (nullable = true)
 |-- _source: struct (nullable = true)
 |    |-- AppName: string (nullable = true)
 |    |-- Contract: string (nullable = true)
 |    |-- Mac: string (nullable = true)
 |    |-- TotalDuration: long (nullable = true)
 |-- _type: string (nullable = true)



In [13]:
df = df.select('_source.AppName', '_source.Contract', '_source.Mac', '_source.TotalDuration')

In [14]:
df.show()

+-------+---------+------------+-------------+
|AppName| Contract|         Mac|TotalDuration|
+-------+---------+------------+-------------+
|  KPLUS|HNH579912|0C96E62FC55C|          254|
|  KPLUS|HUFD40665|CCEDDC333614|         1457|
|  KPLUS|HNH572635|B068E6A1C5F6|         2318|
|  KPLUS|HND141717|08674EE8D2C2|         1452|
|  KPLUS|HNH743103|402343C25D7D|          251|
|  KPLUS|HNH893773|B84DEE76D3B8|          924|
|  KPLUS|HND083642|B84DEE849A0F|         1444|
|  KPLUS|DNFD74404|90324BB44C39|          691|
|  KPLUS|DTFD21200|B84DEED27709|         1436|
|  KPLUS|LDFD05747|0C96E6C95E53|         1434|
|  KPLUS|HNH063566|B84DEEDD1C85|          687|
|  KPLUS|HNH866786|10394E2790A5|          248|
|  KPLUS|NBAAA1128|10394E47C1AF|          247|
|  KPLUS|HNH960439|B84DEED34371|          683|
|  KPLUS|HNJ035736|CCD4A1FA86A5|          246|
|  KPLUS|NTFD93673|B84DEEEF4763|         2288|
|  KPLUS|HNJ063267|10394E172CA7|         2282|
|  KPLUS|HNH790383|4CEBBD53378B|          906|
|  KPLUS|THFD

In [15]:
df.withColumn('Date', lit('2022-04-01')).show(10)

+-------+---------+------------+-------------+----------+
|AppName| Contract|         Mac|TotalDuration|      Date|
+-------+---------+------------+-------------+----------+
|  KPLUS|HNH579912|0C96E62FC55C|          254|2022-04-01|
|  KPLUS|HUFD40665|CCEDDC333614|         1457|2022-04-01|
|  KPLUS|HNH572635|B068E6A1C5F6|         2318|2022-04-01|
|  KPLUS|HND141717|08674EE8D2C2|         1452|2022-04-01|
|  KPLUS|HNH743103|402343C25D7D|          251|2022-04-01|
|  KPLUS|HNH893773|B84DEE76D3B8|          924|2022-04-01|
|  KPLUS|HND083642|B84DEE849A0F|         1444|2022-04-01|
|  KPLUS|DNFD74404|90324BB44C39|          691|2022-04-01|
|  KPLUS|DTFD21200|B84DEED27709|         1436|2022-04-01|
|  KPLUS|LDFD05747|0C96E6C95E53|         1434|2022-04-01|
+-------+---------+------------+-------------+----------+
only showing top 10 rows



In [20]:
df.count() ##Check how many rows in raw data 

1654519

In [19]:
df.select('Mac').distinct().count() ##Check how many unique rows

1498169

In [21]:
df.select('AppName').distinct(). show()

+-------+
|AppName|
+-------+
|  KPLUS|
|  RELAX|
|  CHILD|
|   FIMS|
|CHANNEL|
|  SPORT|
|    VOD|
+-------+



In [22]:
df.select('Contract').distinct().count()

1360622